# Pooled Inference of α and ∆ε

© 2019 Griffin Chure. This work is licensed under a [Creative Commons Attribution License CC-BY 4.0](https://creativecommons.org/licenses/by/4.0/). All code contained herein is licensed under an [MIT license](https://opensource.org/licenses/MIT).

--- 

In [1]:
import sys
sys.path.insert(0, '../../')
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import mwc.bayes
import mwc.stats
import mwc.model
import mwc.viz
colors = mwc.viz.personal_style()
constants = mwc.model.load_constants()

In this notebook, we explore a means to simultaneously infer the calibration factor $\alpha$ and the DNA binding energy from a collection of single-cell measurements. Rather than clustering data into bins of ATC concentration, we will consider fitting the data to all single cell measurements assuming they are drawn from a master distribution.  

To begin, we will load the data and look at all of the calibration factor data pooled together.  